# Введение в обработку текста на естественном языке

Материалы:
* Макрушин С.В. Лекция 9: Введение в обработку текста на естественном языке\
* https://realpython.com/nltk-nlp-python/
* https://scikit-learn.org/stable/modules/feature_extraction.html

## Задачи для совместного разбора

In [4]:
!pip install pymorphy2
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import CountVectorizer
import pandas as pd
import nltk
nltk.download()
from nltk.tokenize import word_tokenize
from nltk.tokenize import sent_tokenize
from nltk.metrics.distance import edit_distance
import random
from nltk.stem import SnowballStemmer
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from scipy.spatial import distance
import pymorphy2
import string

showing info https://raw.githubusercontent.com/nltk/nltk_data/gh-pages/index.xml


1. Считайте слова из файла `litw-win.txt` и запишите их в список `words`. В заданном предложении исправьте все опечатки, заменив слова с опечатками на ближайшие (в смысле расстояния Левенштейна) к ним слова из списка `words`. Считайте, что в слове есть опечатка, если данное слово не содержится в списке `words`. 

In [8]:
text = 'с велечайшим усилием выбравшись из потока убегающих людей Кутузов со свитой уменьшевшейся вдвое поехал на звуки выстрелов русских орудий'
with open ("litw-win.txt", "r", encoding='windows-1251') as f:
    words = [line.strip().split()[1] for line in f]
words[-5:]

['высокопревосходительства',
 'попреблагорассмотрительст',
 'попреблагорассмотрительствующемуся',
 'убегающих',
 'уменьшившейся']

In [12]:
f_lst = []
for i in text.split():
    f_lst.append(min(words, key=lambda w: edit_distance(w, i)))

In [13]:
' '.join(f_lst)

'с величайшим усилием выбравшись из потока убегающих людей кутузов со свитой уменьшившейся вдвое поехал на звуки выстрелов русских орудий'

2. Разбейте текст из формулировки задания 1 на слова; проведите стемминг и лемматизацию слов.

In [ ]:
text3= ( 'Считайте слова из файла litw-win.txt и запишите их в список words.\
В заданном предложении исправьте все опечатки, заменив слова с опечатками на\
ближайшие (в смысле расстояния Левенштейна) к ним слова из списка words. Считайте, что в слове есть опечатка,\
если данное слово не содержится в списке words')
stemmer = nltk.SnowballStemmer('russian')
lemmer = pymorphy2.MorphAnalyzer()
word2  = [[word, stemmer.stem(word), lemmer.parse(word)[0].normal_form] for word in nltk.word_tokenize(text3) if word not in string.punctuation]
df2 = pd.DataFrame(word2, columns=['word', 'stemma', 'lemma'])
df2.head()

3. Преобразуйте предложения из формулировки задания 1 в векторы при помощи `CountVectorizer`.

In [36]:
text3= ( 'Считайте слова из файла litw-win.txt и запишите их в список words.\
В заданном предложении исправьте все опечатки, заменив слова с опечатками на\
ближайшие (в смысле расстояния Левенштейна) к ним слова из списка words. Считайте, что в слове есть опечатка,\
если данное слово не содержится в списке words')
CountVectorizer().fit_transform(nltk.sent_tokenize(text3)).toarray()


array([[1, 1, 1, 2, 1, 0, 0, 0, 1, 1, 1, 2, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1,
        1, 3, 0, 0, 1, 0, 1, 0, 1, 1, 1, 0],
       [0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0,
        0, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1]], dtype=int64)

## Лабораторная работа 9

### Расстояние редактирования

1.1 Загрузите предобработанные описания рецептов из файла `preprocessed_descriptions.csv`. Получите набор уникальных слов `words`, содержащихся в текстах описаний рецептов (воспользуйтесь `word_tokenize` из `nltk`). 

In [7]:
data = pd.read_csv('preprocessed_descriptions.csv')
data['preprocessed_descriptions'] = data['preprocessed_descriptions'].astype(str)
descriptions = ' '.join(list(data['preprocessed_descriptions']))
total_words = list(nltk.word_tokenize(descriptions.lower()))
words = set(nltk.word_tokenize(descriptions.lower()))
print(len(words))



32868


1.2 Сгенерируйте 5 пар случайно выбранных слов и посчитайте между ними расстояние редактирования.

In [8]:
words2 = list(words)
lst = []
for i in range(5):
    w1, w2 = random.sample(words2, 2)
    lst.append(f'{w1} - {w2}. Расстояние редактирования: {nltk.edit_distance(w1, w2)}')
print(*lst, sep = '\n')

differencere - aiken. Расстояние редактирования: 9
kathy - proceeding. Расстояние редактирования: 10
810 - syruptime. Расстояние редактирования: 9
makeno - deficient. Расстояние редактирования: 7
lumpia - plethora. Расстояние редактирования: 6


1.3 Напишите функцию, которая для заданного слова `word` возвращает `k` ближайших к нему слов из списка `words` (близость слов измеряется с помощью расстояния Левенштейна)

In [9]:
def distance(word, k):
    words = ['яблоко', 'банан', 'апельсин','apple', 'груша', 'киви', 'ананас', 'манго', 'арбуз', 'слива',\
         'виноград', 'гранат', 'лимонад', 'персиммон', 'крыжовник', 'голубика', 'бананчик', 'апельсинчик', 'аппак','ааппельсин']
    d = {i:edit_distance(word, i) for i in words if i != word}
    print(d)
    lst = list(sorted(d.values()))[:k]
    f_lst = []
    for i in lst:
        for key,v in d.items():
            if v == i:
                f_lst.append(f'{word} - {key}. Близость слов: {v}')
            if len(f_lst) == k:
                break
    return f_lst
distance('апельсин', 3)

{'яблоко': 7, 'банан': 7, 'apple': 8, 'груша': 8, 'киви': 7, 'ананас': 6, 'манго': 8, 'арбуз': 7, 'слива': 7, 'виноград': 8, 'гранат': 8, 'лимонад': 8, 'персиммон': 6, 'крыжовник': 8, 'голубика': 7, 'бананчик': 7, 'апельсинчик': 3, 'аппак': 6, 'ааппельсин': 2}


['апельсин - ааппельсин. Близость слов: 2',
 'апельсин - апельсинчик. Близость слов: 3',
 'апельсин - ананас. Близость слов: 6']

### Стемминг, лемматизация

2.1 На основе результатов 1.1 создайте `pd.DataFrame` со столбцами: 
    * word
    * stemmed_word 
    * normalized_word 

Столбец `word` укажите в качестве индекса. 

Для стемминга воспользуйтесь `SnowballStemmer`, для нормализации слов - `WordNetLemmatizer`. Сравните результаты стемминга и лемматизации.

In [10]:
stemmer = SnowballStemmer('english')
lemmatizer = WordNetLemmatizer()
df = pd.DataFrame(words, columns = ['word'])
df['stemmed_word'] = df.apply(lambda x: stemmer.stem(x['word']), axis = 1)
df['normalized_word'] = df.apply(lambda x: lemmatizer.lemmatize(x['word']), axis = 1)
df = df.set_index('word')
df

,stemmed_word,normalized_word
word,,
variation,variat,variation
lora,lora,lora
starter,starter,starter
skimp,skimp,skimp
sabra,sabra,sabra
...,...,...
septoct,septoct,septoct
kibbeh,kibbeh,kibbeh
itnote,itnot,itnote


2.2. Удалите стоп-слова из описаний рецептов. Какую долю об общего количества слов составляли стоп-слова? Сравните топ-10 самых часто употребляемых слов до и после удаления стоп-слов.

In [11]:
from nltk.corpus import stopwords

stop_words = set(stopwords.words('english'))
data['preprocessed_descriptions'] = data['preprocessed_descriptions'].\
apply(lambda x: ' '.join([word for word in x.split() if word not in stop_words]))
words3 = [word for word in total_words if word not in stop_words]
print('Доля от общего кол-ва, которую составляли стоп слова:', 1 - len(words3)/len(total_words))
data

Доля от общего кол-ва, которую составляли стоп слова: 0.45646494716721886


,Unnamed: 0,name,preprocessed_descriptions
0,0,george s at the cove black bean soup,original recipe created chef scott meskan geor...
1,1,healthy for them yogurt popsicles,children friends ask homemade popsicles mornin...
2,2,i can t believe it s spinach,go surprised even
3,3,italian gut busters,sisterinlaw made us family get together delici...
4,4,love is in the air beef fondue sauces,think fondue romantic casual dinner wonderful ...
...,...,...,...
29995,29995,zurie s holey rustic olive and cheddar bread,based french recipe changed substantially warn...
29996,29996,zwetschgenkuchen bavarian plum cake,traditional fresh plum cake thought originated...
29997,29997,zwiebelkuchen southwest german onion cake,traditional late summer early fall snack usual...
29998,29998,zydeco soup,delicious soup originally found better homes g...


In [12]:
print('до удаления стоп-слов')
for k, v in nltk.FreqDist(total_words).most_common(10):
    print(f'{k} - {v}')
print('---------------------')
print('после удаления стоп-слов')
for k, v in nltk.FreqDist(words3).most_common(10):
    print(f'{k} - {v}')

до удаления стоп-слов
the - 40072
a - 34951
and - 30245
this - 26859
i - 24836
to - 23471
is - 20285
it - 19756
of - 18364
for - 15939
---------------------
после удаления стоп-слов
recipe - 14871
make - 6326
time - 5137
use - 4620
great - 4430
like - 4167
easy - 4152
one - 3872
made - 3810
good - 3791


### Векторное представление текста

3.1 Выберите случайным образом 5 рецептов из набора данных. Представьте описание каждого рецепта в виде числового вектора при помощи `TfidfVectorizer`

In [13]:
df31 = data.sample(n=5)
vectorizer = TfidfVectorizer()
vectors = vectorizer.fit_transform(df31['preprocessed_descriptions'])
for i, vector in enumerate(vectors):
    print(f"Рецепт {i+1}\n")
    print(vector.toarray(), '\n')  

Рецепт 1

[[0.         0.         0.25136527 0.         0.31156077 0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.25136527 0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.31156077 0.         0.31156077 0.
  0.         0.31156077 0.         0.         0.         0.31156077
  0.         0.31156077 0.         0.         0.31156077 0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.31156077 0.         0.         0.
  0.         0.31156077]] 

Рецепт 2

[[0.         0.         0.20329067 0.         0.         0.
  0.         0.         0.         0.25197355 0.         0.25197355
  0.25197355 0.25197355 0.25197355 0.25197355 0.         0.
  0.         0.         0.         0.16874962 0.         0.
  0.         0.         0.20329067 0

3.2 Вычислите близость между каждой парой рецептов, выбранных в задании 3.1, используя косинусное расстояние (`scipy.spatial.distance.cosine`) Результаты оформите в виде таблицы `pd.DataFrame`. В качестве названий строк и столбцов используйте названия рецептов.

In [14]:
from scipy.spatial.distance import cosine

df31['preprocessed_descriptions'] = df31['preprocessed_descriptions'].astype(str)
vectorizer = TfidfVectorizer()
vectors = vectorizer.fit_transform(df31['preprocessed_descriptions']).toarray()
df32 = pd.DataFrame(index = df31['name'], columns = df31['name'])
for i, r1 in enumerate(df31['name']):
    for j, r2 in enumerate(df31['name']):
        df32.at[r1, r2] = 1 - cosine(vectors[i], vectors[j])
df32 = df32.reset_index().rename(columns={'name': ' '})
df32 = df32.set_index(' ')
df32

name,italian vegetable pancakes,cheezy chicken parmesan with zucchini pasta,easy cherry pie bars,chicken sate,huckleberry rhubarb jam
,,,,,
italian vegetable pancakes,1,0.1022,0.0,0.0,0.0
cheezy chicken parmesan with zucchini pasta,0.1022,1,0.0,0.021189,0.060858
easy cherry pie bars,0.0,0.0,1,0.0,0.0
chicken sate,0.0,0.021189,0.0,1,0.045282
huckleberry rhubarb jam,0.0,0.060858,0.0,0.045282,1


3.3 Какие рецепты являются наиболее похожими? Прокомментируйте результат (словами).

In [16]:
df33 = df32.where(df32 != 1).max().max()
print('Наиболее похожие рецепты: italian vegetable pancakes и cheezy chicken parmesan with zucchini pasta')
df33

Наиболее похожие рецепты: italian vegetable pancakes и cheezy chicken parmesan with zucchini pasta


0.10220042776744487